In [1]:
import numpy as np
from pathlib import Path

# -----------------------------------------------------------------------------
# UNIFIED CONFIGURATION
# -----------------------------------------------------------------------------

# --- Paths ---
BASE_DIR = Path('/project/dinner/ianjefab/papers/202508_ELP/data')
SIM_DIR = BASE_DIR / 'sim_data' / 'sim_spec'      
EXP_DIR = BASE_DIR / 'exp_data' / 'processed_FTIRs'  
WAXIS_FILE = BASE_DIR / 'waxis.txt'               
OUTPUT_DIR = Path('./overlap_informative')      

# --- Data & Labels ---
FORCE_FIELDS = ['CHARMM27_TIP3P', 'CHARMM36m_TIP3P']  

# ONLY THE C-TURN INFORMATIVE ISOTOPES
ISOTOPE_SYSTEM_NAMES = ['V1', 'V1G3V4', 'V1V4', 'V4']  
ISOTOPE_FILE_NAME_MAP = {  # Maps system names to experimental file naming convention
    'V1': 'V1', 'V1G3V4': 'V1G3V4', 'V1V4': 'V1V4', 'V4': 'V4',
}
MAP_TYPE = 'DC1Fnew'  # Map name

# --- Analysis Parameters ---
WAVENUMBER_RANGE = (1525, 1750)
SHIFT_VALUES = [-2, 0, 2]        # Test systematic frequency shifts  
THETA_RANGE = np.logspace(-1, 4, 1000)  # Shape: (1000,) - Reweighting parameters 0.1 to 10000
NUM_SPECTRA = 5000               # Number of segments

# ONLY THE INFORMATIVE REGIONS
INFORMATIVE_RANGES = {
    'V1': (1533.8, 1588.8),
    'V1G3V4': (1536.8, 1605.8),
    'V1V4': (1535.8, 1582.8),
    'V4': (1572.8, 1584.8),
}

# -----------------------------------------------------------------------------
# UTILITY & ANALYSIS FUNCTIONS
# -----------------------------------------------------------------------------

def load_wavenumbers():
    """Loads and trims the reference wavenumber axis.
    
    Returns:
        np.array: Wavenumber axis trimmed to WAVENUMBER_RANGE, shape (~225,)
    """
    waxis = np.loadtxt(WAXIS_FILE)  # Full wavenumber axis, shape (n_points,)
    mask = (waxis >= WAVENUMBER_RANGE[0]) & (waxis <= WAVENUMBER_RANGE[1])
    return waxis[mask]  # Trimmed axis, shape (225,) 

def normalize_by_area(spectrum, waxis):
    """Normalizes a spectrum by its integrated area.
    Args:
        spectrum: FTIR intensity values, shape (n_wavenumbers,)
        waxis: Wavenumber axis, shape (n_wavenumbers,)
    
    Returns:
        np.array: Area-normalized spectrum, same shape as input
    """
    area = np.trapz(spectrum, waxis)  # Integrate using trapezoidal rule
    return spectrum / area 

def shift_spectrum(spectrum, shift_amount, waxis):
    """Shifts a spectrum along the wavenumber axis by a given amount
    to simulate the systematic error
    Args:
        spectrum: FTIR spectrum, shape (n_wavenumbers,)
        shift_amount: Shift in cm⁻¹ (positive = higher frequency)
        waxis: Wavenumber axis, shape (n_wavenumbers,)
    
    Returns:
        np.array: Shifted spectrum, same shape as input
    """
    if shift_amount == 0:
        return spectrum.copy()
    
    spacing = waxis[1] - waxis[0]  # Wavenumber spacing (cm⁻¹/point)
    num_indices = int(round(abs(shift_amount) / spacing))  # Convert cm⁻¹ to array indices
    shifted = np.zeros_like(spectrum)
    
    if shift_amount < 0:  # Left shift (lower frequency)
        shifted[:-num_indices] = spectrum[num_indices:]
    else:  # Right shift (higher frequency)  
        shifted[num_indices:] = spectrum[:-num_indices]
    return shifted

def calculate_overlap_error(experimental, simulated, waxis, thetas):
    """Calculates overlap-based error between experimental and simulated spectra.
    The overlap coefficient measures spectral similarity (1 = perfect match, 0 = no overlap).
    Error = (1 - overlap) gives a penalty that decreases with better agreement.
    Args:
        experimental: Reference experimental spectrum, shape (n_wavenumbers,)
        simulated: Simulated spectrum from MD, shape (n_wavenumbers,)  
        waxis: Wavenumber axis, shape (n_wavenumbers,)
        thetas: Reweighting parameters, shape (1000,)
    Returns:
        np.array: Error values for each theta, shape (1000,)
                 Lower error = better match to experiment
    """
    # Calculate overlap coefficient 
    numerator = np.trapz(experimental * simulated, waxis)  
    exp_norm = np.sqrt(np.trapz(experimental**2, waxis))   
    sim_norm = np.sqrt(np.trapz(simulated**2, waxis))     
    
    overlap = numerator / (exp_norm * sim_norm) if (exp_norm * sim_norm) > 0 else 0
    # overlap \in [0,1]: 1 = identical spectra, 0 = orthogonal spectra
    
    error = (1 - overlap)  # Convert to error: 0 = perfect, 1 = worst
    return error * thetas   # Scale by theta values, shape (1000,)

def load_all_experimental_data(waxis_full):
    """Loads and normalizes all experimental spectra into a dictionary.
    
    Args:
        waxis_full: Wavenumber axis, shape (n_wavenumbers,)
    
    Returns:
        dict: {'isotope_name': normalized_spectrum, ...}
              Each spectrum has shape (n_wavenumbers,)
    """
    print("Loading experimental data...")
    exp_data = {}
    for iso, file_part in ISOTOPE_FILE_NAME_MAP.items():
        fname = f'GVGn1_{file_part}_50mgmL_phos_150mM_pH1_basecorr_processed.npy'
        spectrum = np.load(EXP_DIR / fname)  # Shape: (n_wavenumbers,)
        exp_data[iso] = normalize_by_area(spectrum, waxis_full)
    print("Experimental data loaded.")
    return exp_data

# -----------------------------------------------------------------------------
# MAIN EXECUTION
# -----------------------------------------------------------------------------

def main():
    """Main function to run the complete error calculation pipeline.
    For each combination of force field, isotope, and shift value:
    1. Load experimental reference spectrum
    2. Load 5000 simulated spectra from MD
    3. Apply wavenumber shift to simulated spectra  
    4. Extract isotope-specific informative region
    5. Calculate overlap error vs experimental for each MD frame (informative region only)
    6. Save error matrix: (5000 frames, 1000 theta values)
    
    Output files are used by reweighting analysis.
    """
    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
    
    waxis_full = load_wavenumbers()  # Shape: (~225,)
    experimental_data = load_all_experimental_data(waxis_full)  # Dict of 4 reference spectra

    # Create a dictionary of boolean masks for each isotope's informative region
    mask_dict = {
        iso: (waxis_full >= r_min) & (waxis_full <= r_max)
        for iso, (r_min, r_max) in INFORMATIVE_RANGES.items()
    }  # Each mask has shape (225,) with True for informative region

    # Process all combinations: 2 force fields × 4 isotopes × 3 shifts = 24 output files
    for ff in FORCE_FIELDS:
        print(f"\n--- Processing Force Field: {ff} ---")
        for iso in ISOTOPE_SYSTEM_NAMES:
            print(f"  -> Isotope: {iso}")
            try:
                # Load simulated data
                sim_file = SIM_DIR / ff / f'{MAP_TYPE}_{ff}_{iso}_ftir.npy'
                sim_raw = np.load(sim_file)  # Shape: (5000, n_wavenumbers)
                exp_spectrum = experimental_data[iso]  # Shape: (n_wavenumbers,)
                mask = mask_dict[iso]  # Boolean mask for this isotope's informative region
                waxis_masked = waxis_full[mask]  # Extract wavenumber axis for informative region

                # Process each wavenumber shift (-2, 0, +2 cm⁻¹)
                for shift in SHIFT_VALUES:
                    print(f"    -> Shift: {shift} cm⁻¹")
                    errors = np.zeros((NUM_SPECTRA, len(THETA_RANGE)))  # Shape: (5000, 1000)
                    
                    # Calculate error for each segment
                    for i in range(NUM_SPECTRA):
                        # Shift first, then normalize the full spectrum
                        sim_shifted = shift_spectrum(sim_raw[i], shift, waxis_full)  # Shape: (n_wavenumbers,)
                        sim_norm = normalize_by_area(sim_shifted, waxis_full)  # Shape: (n_wavenumbers,)
                        
                        # Apply mask to both spectra before calculating error
                        exp_masked = exp_spectrum[mask]  # Shape: (n_wavenumbers_informative,)
                        sim_masked = sim_norm[mask]      # Shape: (n_wavenumbers_informative,)
                        
                        errors[i] = calculate_overlap_error(
                            exp_masked, sim_masked, waxis_masked, THETA_RANGE
                        )  # Shape: (1000,) - one error per theta value
                    
                    # Save error matrix for this combination
                    output_file = OUTPUT_DIR / f'error_overlap_{iso}_{ff}_ch27_{shift}_C.npy'
                    np.save(output_file, errors)  # Shape: (5000, 1000)
                    print(f"Saved error matrix to: {output_file.name}")

            except FileNotFoundError as e:
                print(f"ERROR: Could not find input file. Skipping. ({e})")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")

# RUN
if __name__ == "__main__":
    main()

Loading experimental data...
Experimental data loaded.

--- Processing Force Field: CHARMM27_TIP3P ---
  -> Isotope: V1
    -> Shift: -2 cm⁻¹
Saved error matrix to: error_overlap_V1_CHARMM27_TIP3P_ch27_-2_C.npy
    -> Shift: 0 cm⁻¹
Saved error matrix to: error_overlap_V1_CHARMM27_TIP3P_ch27_0_C.npy
    -> Shift: 2 cm⁻¹
Saved error matrix to: error_overlap_V1_CHARMM27_TIP3P_ch27_2_C.npy
  -> Isotope: V1G3V4
    -> Shift: -2 cm⁻¹
Saved error matrix to: error_overlap_V1G3V4_CHARMM27_TIP3P_ch27_-2_C.npy
    -> Shift: 0 cm⁻¹
Saved error matrix to: error_overlap_V1G3V4_CHARMM27_TIP3P_ch27_0_C.npy
    -> Shift: 2 cm⁻¹
Saved error matrix to: error_overlap_V1G3V4_CHARMM27_TIP3P_ch27_2_C.npy
  -> Isotope: V1V4
    -> Shift: -2 cm⁻¹
Saved error matrix to: error_overlap_V1V4_CHARMM27_TIP3P_ch27_-2_C.npy
    -> Shift: 0 cm⁻¹
Saved error matrix to: error_overlap_V1V4_CHARMM27_TIP3P_ch27_0_C.npy
    -> Shift: 2 cm⁻¹
Saved error matrix to: error_overlap_V1V4_CHARMM27_TIP3P_ch27_2_C.npy
  -> Isotope: V